In [1]:
import pandas as pd
from urllib.request import urlopen
import json
import numpy as np

In [2]:
varcd_cod = "0012141"
url = f"https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd={varcd_cod}&lang=PT"
response = urlopen(url)
data_json = json.loads(response.read())

In [3]:
data_json

[{'IndicadorCod': '0012141',
  'IndicadorDsg': 'Indivíduos a residir sozinhos (N.º) por Local de residência à data dos Censos [2021] (NUTS - 2013), Sexo e Grupo etário; Decenal - INE, Recenseamento da população e habitação - Censos 2021',
  'MetaInfUrl': 'https://www.ine.pt/bddXplorer/htdocs/minfo.jsp?var_cd=0012141&lingua=PT',
  'DataExtracao': '2024-06-18T09:03:39.557+01:00',
  'DataUltimoAtualizacao': '2024-01-18',
  'UltimoPref': '2021',
  'Dados': {'2021': [{'geocod': '0101',
     'geodsg': 'Águeda',
     'dim_3': '1',
     'dim_3_t': 'H',
     'dim_4': '01',
     'dim_4_t': 'Menos de 20 anos',
     'ind_string': '1',
     'valor': '1'},
    {'geocod': '010103',
     'geodsg': 'Aguada de Cima',
     'dim_3': '1',
     'dim_3_t': 'H',
     'dim_4': '02',
     'dim_4_t': '20 - 24 anos',
     'ind_string': '1',
     'valor': '1'},
    {'geocod': '010103',
     'geodsg': 'Aguada de Cima',
     'dim_3': '2',
     'dim_3_t': 'M',
     'dim_4': '02',
     'dim_4_t': '20 - 24 anos',
     

In [4]:
freguesias = """
União das freguesias de Aldoar, Foz do Douro e Nevogilde
União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória
União das freguesias de Lordelo do Ouro e Massarelos
Bonfim
Campanhã
Paranhos
Ramalde
""".strip().split("\n")
freguesias

['União das freguesias de Aldoar, Foz do Douro e Nevogilde',
 'União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória',
 'União das freguesias de Lordelo do Ouro e Massarelos',
 'Bonfim',
 'Campanhã',
 'Paranhos',
 'Ramalde']

In [5]:
l1 = data_json[0]["Dados"]["2021"]
l2 = filter(lambda x: x["geodsg"] in freguesias, l1)
l3 = set([(x["geocod"], x["geodsg"]) for x in l2])
l3

{('091208', 'Paranhos'),
 ('131202', 'Bonfim'),
 ('131203', 'Campanhã'),
 ('131210', 'Paranhos'),
 ('131211', 'Ramalde'),
 ('131216', 'União das freguesias de Aldoar, Foz do Douro e Nevogilde'),
 ('131217',
  'União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória'),
 ('131218', 'União das freguesias de Lordelo do Ouro e Massarelos')}

In [6]:
datas_jsons = []
for code, name in l3:
    varcd_cod = '0012373'
    url = f"https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd={varcd_cod}&Dim2={code}&lang=PT"
    response = urlopen(url)
    datas_jsons.append(json.loads(response.read()))
"done"

'done'

In [7]:
datas_jsons

[[{'IndicadorCod': '0012373',
   'IndicadorDsg': 'População residente que vive no alojamento a maior parte do ano (N.º) por Local de residência à data dos Censos [2021] (NUTS - 2024), Sexo, Condição perante o trabalho, Escalão de duração dos movimentos pendulares e Local de trabalho ou estudo; Decenal - INE, Recenseamento da população e habitação - Censos 2021',
   'MetaInfUrl': 'https://www.ine.pt/bddXplorer/htdocs/minfo.jsp?var_cd=0012373&lingua=PT',
   'DataExtracao': '2024-06-18T09:03:48.682+01:00',
   'DataUltimoAtualizacao': '2024-03-19',
   'UltimoPref': '2021',
   'Dados': {'2021': [{'geocod': '131216',
      'geodsg': 'União das freguesias de Aldoar, Foz do Douro e Nevogilde',
      'dim_3': '1',
      'dim_3_t': 'H',
      'dim_4': '1',
      'dim_4_t': 'Empregada',
      'dim_5': '1',
      'dim_5_t': 'Até 15 minutos',
      'dim_6': '40107',
      'dim_6_t': 'Espinho',
      'ind_string': '1',
      'valor': '1'},
     {'geocod': '131216',
      'geodsg': 'União das fregues

In [8]:
final_movement_json = []
for data in datas_jsons:
    l_movement = data[0]["Dados"]['2021']
    l_movement = list(filter(lambda x: x["dim_5"] == "T" and "freguesia" in x["dim_6_t"], l_movement))
    final_movement_json += l_movement
final_movement = pd.DataFrame(final_movement_json)
final_movement = final_movement.drop(columns=["geocod","dim_3", "dim_4", "dim_5", "dim_5_t", "dim_6", "ind_string"])
final_movement = final_movement.rename(columns={"geodsg":"neighborhood","dim_3_t":"sex", "dim_4_t":"occupation" ,"dim_6_t":"destination", "valor":"value"})

for neighborhood in final_movement["neighborhood"].unique():
    total = final_movement[(final_movement["neighborhood"] == neighborhood) & (final_movement["sex"] == "HM") & (final_movement["occupation"] == "Total")]
    final_movement.loc[len(final_movement)] = [neighborhood, "HM", "Total", "Total", str(total["value"].apply(int).sum())]

final_movement = final_movement.sort_values(by=["neighborhood"],ignore_index=True)

final_movement.to_csv("data_out/movimentos_porto_por_freguesia.csv", index=False)
final_movement

,neighborhood,sex,occupation,destination,value
0,Bonfim,HM,Empregada,Na freguesia onde reside atualmente,1680
1,Bonfim,M,Estudante,"No mesmo município, noutra freguesia",680
2,Bonfim,M,Estudante,Na freguesia onde reside atualmente,701
3,Bonfim,H,Estudante,"No mesmo município, noutra freguesia",702
4,Bonfim,H,Empregada,Na freguesia onde reside atualmente,756
...,...,...,...,...,...
146,União das freguesias de Lordelo do Ouro e Mass...,H,Estudante,"No mesmo município, noutra freguesia",1255
147,União das freguesias de Lordelo do Ouro e Mass...,M,Estudante,"No mesmo município, noutra freguesia",1196
148,União das freguesias de Lordelo do Ouro e Mass...,H,Estudante,Na freguesia onde reside atualmente,986
149,União das freguesias de Lordelo do Ouro e Mass...,M,Total,Na freguesia onde reside atualmente,2135


In [16]:
new = []
neighborhoods = final_movement["neighborhood"].unique()
for neighborhood in neighborhoods:
    neighborhoodDF = final_movement[final_movement["neighborhood"] == neighborhood]
    otherNeighborhoods = list(neighborhoods)
    otherNeighborhoods.remove(neighborhood)
    for _, row in neighborhoodDF.iterrows():
        
        total = int(neighborhoodDF[neighborhoodDF["destination"] == "Total"]["value"].iloc[0])

        proportion = int(row["value"])/total

        if row["destination"] == "No mesmo município, noutra freguesia":
            count = int(row["value"])/len(otherNeighborhoods)
            proportion = count/total
            for other in otherNeighborhoods:
                new.append((row["neighborhood"], row["sex"], row["occupation"], other, count, proportion))
        elif row["destination"] == "Na freguesia onde reside atualmente":
            new.append((row["neighborhood"], row["sex"], row["occupation"], neighborhood, row["value"], proportion))
        else:
            new.append((row["neighborhood"], row["sex"], row["occupation"], row["destination"], row["value"], proportion))
        

commutes = pd.DataFrame(new, columns=["neighborhood","sex","occupation","destination","value", "probability"])
commutes.to_csv("data_out/commutes.csv", index=False, float_format="%g")

In [103]:
# #OD Matrix
# simplified = final_movement[(final_movement["tempo"]=="Total") & (final_movement["genero"]=="HM") & (final_movement["ocupacao"]=="Total")].drop(columns=["genero","ocupacao","tempo"])

# mapping = {v:k for k,v in enumerate(simplified["freguesia"].unique())}
# mappingRev = {k:v for k,v in enumerate(simplified["freguesia"].unique())}

# simplified["destino"] = simplified["destino"].apply(lambda x: "same" if x=="Na freguesia onde reside atualmente" else "other")
# simplified["freguesia"] = simplified["freguesia"].apply(lambda x: mapping[x])
# simplified = simplified.rename(columns={"freguesia":"origin", "destino":"destination", "valor":"count"})
# simplified["count"] = simplified["count"].apply(int)
# neighborhoods = simplified['origin'].unique()
# n = len(neighborhoods)

# od_matrix = np.zeros((n, n))

# for index, row in simplified[simplified['destination'] == 'same'].iterrows():
#     od_matrix[row['origin'], row['origin']] = row['count']

# for index, row in simplified[simplified['destination'] == 'other'].iterrows():
#     origin = row['origin']
#     count = row['count']
#     other_neighborhoods = np.delete(neighborhoods, origin)
#     distributed_count = count / len(other_neighborhoods)
#     for destination in other_neighborhoods:
#         od_matrix[origin, destination] = distributed_count

# od_matrix_df = pd.DataFrame(od_matrix, index=neighborhoods, columns=neighborhoods)
# od_matrix_df.to_csv("data_out/od_matrix.csv", )

# od_long_df = od_matrix_df.stack().reset_index()
# od_long_df.columns = ['origin', 'destination', 'count']

# total_exits = od_long_df.groupby('origin')['count'].sum().reset_index()
# total_exits.columns = ['origin', 'total_exits']


# od_long_df = od_long_df.merge(total_exits, on='origin')
# od_long_df['probability'] = od_long_df['count'] / od_long_df['total_exits']
# od_long_df = od_long_df.drop(columns=['total_exits'])

# od_long_df["count"] = od_long_df["count"].apply(round)
# od_long_df["origin"] = od_long_df["origin"].apply(lambda x: mappingRev[x])
# od_long_df["destination"] = od_long_df["destination"].apply(lambda x: mappingRev[x])
# od_long_df.to_csv("data_out/commutes.csv", index=False)
# od_long_df